# TO DO

- Make history hashes work with strings of commands


### Problems:

- eval has problems with equal signs
- random initializations will cause problems w/ hash_history
    - FOr now: if there's "random" in the code string, raise an Error
- Would we run into issues w/ ops like matmul if EmptyActionObjects can't infer size/shape b/c of missing metadata/context?

In [1]:
from syft.core.node.new.action_object import ActionObject

In [2]:
import numpy as np
private_dataset = 2 * np.random.random((10, 10)) - 1  # pretend this is on a domain node

In [11]:
priv = ActionObject(syft_action_data=private_dataset)

Lights... Cameras... ACTION OBJECT!


In [13]:
priv.transpose()

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: 2343825891455800785

In [14]:
a = ActionObject(id=priv.id, syft_result_obj=np.array([]))

Lights... Cameras... ACTION OBJECT!


In [15]:
a.transpose()

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: 2343825891455800785

In [5]:
try:
    eval("x = 1")
except SyntaxError:
    print("Eval has problems with equality")

Eval has problems with equality


In [51]:
def SingleCallPolicy(*args, **kwargs):
    print("This lets you run the code once")
    return 1

def code_request_policy(*args, **kwargs):
    print("PySyft is currently performing code verification")
    if "policy" in kwargs:
        print(f'Policy in effect: {kwargs["policy"]}')
    else:
        print("No policy specified, using SingleCallPolicy by default")
        
    assert ("downloadable_output_args" in kwargs and kwargs["downloadable_output_args"] is not None), "Must specify result you want"
    
    
    def wrapper(func):
        return func
    print("Verified! Sending to Data Owner.")
    return wrapper

In [53]:
@code_request_policy(policy=SingleCallPolicy,
                        parties_data=private_dataset,
                        fancy_string=str,  # this is where we'll be chucking in code
                        downloadable_output_args=["output"])
def my_analysis(parties_data):
    import numpy as np
    
    def relu(x):
        return x * (x > 0)
    
    W = np.random.random((3, 10))
    b = np.random.random((10,))
    x = W @ parties_data + b
    output = relu(x)
    return output

PySyft is currently performing code verification
Policy in effect: <function SingleCallPolicy at 0x7f23b4c7e550>
Verified! Sending to Data Owner.


In [28]:
my_analysis(private_dataset)

array([[-0.        ,  1.20330984, -0.        ,  1.18518865,  0.85789389,
        -0.        ,  1.8746026 ,  0.40868893, -0.        , -0.        ],
       [-0.        ,  1.49154832, -0.        ,  1.23281906,  0.00745984,
        -0.        ,  1.95799098,  0.44815021, -0.        ,  0.90847315],
       [-0.        ,  0.94680866, -0.        ,  1.64700006,  1.3487248 ,
        -0.        ,  1.12224282, -0.        , -0.        , -0.        ]])

In [54]:
final_code_block = """import numpy as np
def relu(x):
    return x * (x > 0)

W = np.random.random((3, 10))
b = np.random.random((10,))
x = W @ private_dataset + b
output = relu(x)
"""

In [72]:
easy_code_block = """result = private_dataset + 1"""

In [73]:
inputs = [private_dataset]
outputs = [result]

In [74]:
a.id

<UID: ee17296e96114b9980d0fd0d0f9cedc6>

In [75]:
easy_code_block.replace("result", "potato")

'potato = private_dataset + 1'

In [78]:
t = ActionObject(id=a.id)

Lights... Cameras... ACTION OBJECT!


In [77]:
t + 6

AttributeError: 'NoneType' object has no attribute '__add__'